In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib
import sys, re
from os import listdir
from os.path import isfile, join

import torch
import numpy as np
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

In [2]:
# if this cell prints "Running on cpu", you must switch runtime environments
# go to Runtime > Change runtime type > Hardware accelerator > GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


In [3]:
def read_embeddings(filename, vocab_size=10000):
    # get the embedding size from the first embedding
    with open(filename, encoding="utf-8") as file:
        word_embedding_dim = len(file.readline().split(" ")) - 2

    vocab = {}

    embeddings = np.zeros((vocab_size, word_embedding_dim))

    with open(filename, encoding="utf-8") as file:
        for idx, line in enumerate(file):
#             if idx + 2 >= vocab_size:
            if idx >= vocab_size:
                break
            cols = line.rstrip().split(" ")
            val = np.array(cols[1:])
            word = cols[0]
#             embeddings[idx + 2] = val
#             vocab[word] = idx + 2
            embeddings[idx] = val
            vocab[word] = idx

    # a FloatTensor is a multidimensional matrix
    # that contains 32-bit floats in every entry
    # https://pytorch.org/docs/stable/tensors.html
    return torch.FloatTensor(embeddings), vocab

In [8]:
# this loads the 10,000 most common word 50-dimensional embeddings
vocab_size = 10000
embeddings, vocab = read_embeddings('/gigaword_chn.all.a2b.uni.ite50.txt', vocab_size)
# print(vocab)

In [9]:
class Dataset():
    def __init__(self, filename):
        
        self.sentences, self.tags = self.read_data(filename)
        self.sentences_chunk = []
        self.tags_chunk = []


    def read_data(self, book):
        """
        Utility function, loads text file into a list of sentence and tag strings

        Arguments:
        - filename:     path to file
        we assume each line is formatted as "<word>\t<tag>\n"

        Returns:
        - sentences:    a list of sentences, where each sentence is a list 
                        words (strings)
        - tags:         a list of tags for each sentence, where tags[i] contains
                        a list of tags (strings) that correspond to the words in 
                        sentences[i]
        """
        sentences = []
        tags = []

        current_sentence = []
        current_tags = []

        book = book.split("\n")
        for line in book:
            if line == "":
                if len(current_sentence) != 0:
                    sentences.append(current_sentence)
                    tags.append(current_tags)
                    
                current_sentence = []
                current_tags = []
            else:
                columns = line.rstrip().split('\t')
                word = columns[0].lower()
                tag = columns[1]
                
                current_sentence.append(word)
                current_tags.append(tag)
        return sentences, tags
    
        

    def get_batches(self, batch_size, vocab, tagset, start, end, omit):
        """

        Batches the data into mini-batches of size `batch_size`

        Arguments:
        - batch_size:       the desired output batch size
        - vocab:            a dictionary mapping word strings to indices
        - tagset:           a dictionary mapping tag strings to indices

        Outputs:

        if is_labeled=True:
        - batched_word_indices:     a list of matrices of dimension (batch_size x max_seq_len)
        - batched_tag_indices:      a list of matrices of dimension (batch_size x max_seq_len)
        - batched_lengths:          a list of arrays of length (batch_size)


        Description: 

        This function partitions the data into batches of size batch_size. If the number
        of sentences in the document is not an even multiple of batch_size, the final batch
        will contain the remaining elements. For example, if there are 82 sentences in the 
        dataset and batch_size=32, we return a list containing two batches of size 32 
        and one final batch of size 18.

        batched_word_indices[b] is a (batch_size x max_seq_len) matrix of integers, 
        containing index representations for sentences in the b-th batch in the document. 
        The `vocab` dictionary provides the correct mapping from word strings to indices. 
        If a word is not in the vocabulary, it gets mapped to UNKNOWN_INDEX (1).
        `max_seq_len` is the maximum sentence length among the sentences in the current batch, 
        which will vary between different batches. All sentences shorter than max_seq_len 
        should be padded on the right with PAD_INDEX (0).

        If the document is labeled, we also batch the document's tags. Analogous to 
        batched_word_indices, batched_tag_indices[b] contains the index representation
        for the tags corresponding to the sentences in the b-th batch  in the document. 
        The `tagset` dictionary provides the correct mapping from tag strings to indicies. 
        All tag lists shorter than `max_seq_len` are padded with IGNORE_TAG_INDEX (-100).

        batched_lengths[b] is a vector of length (batch_size). batched_lengths[b][i] 
        contains the original sentence length *before* padding for the i-th sentence
        in the currrent batch. 

        """
        PAD_INDEX = 0             # reserved for padding words
        UNKNOWN_INDEX = 1         # reserved for unknown words
        IGNORE_TAG_INDEX = -100   # reserved for padding tags
       
        batched_word_indices = []
        batched_tag_indices = []
        batched_lengths = []
        
        if omit == -1: # for dev sets
            sentences = self.sentences_chunk[start:end][0]
            tags = self.tags_chunk[start:end][0]
#             print(len(sentences))
        else: # for training sets
            sentences = [self.sentences_chunk[i] for i in range(start, end) if i != omit]
#             print([ i for i in range(start, end) if i != omit])
            tags = [self.tags_chunk[i] for i in range(start, end) if i != omit]
            
            temp_sen = []
            temp_tag = []
            for i in range(1, len(sentences)):
                temp_sen += sentences[i]
                temp_tag += tags[i]
            sentences = temp_sen
            tags = temp_tag
#             print(len(sentences))
#             print(len(tags))
            
        for num_batch in range(math.ceil(len(sentences) / batch_size)):
            sentence_list = np.array(sentences[num_batch * batch_size : min((num_batch + 1) * batch_size, len(sentences))])
            #batched_lengths
            length_array = np.zeros(len(sentence_list))
            #batched_word_indices
            max_seq_len = len(max(sentence_list, key=len))
            matrix = np.zeros((min(batch_size, len(sentence_list)), max_seq_len))
            for i in range(len(sentence_list)):
                matrix[i] = [vocab[word] if word in vocab else UNKNOWN_INDEX for word in sentence_list[i]] + [PAD_INDEX for i in range(max_seq_len - len(sentence_list[i]))]
                length_array[i] = len(sentence_list[i])
            batched_word_indices.append(matrix)
            batched_lengths.append(length_array)


        #batched_tag_indices
        for num_batch in range(math.ceil(len(tags) / batch_size)):
            tag_list = np.array(tags[num_batch * batch_size : min((num_batch + 1) * batch_size, len(tags))])
            max_seq_len = len(max(tag_list, key=len))
            matrix = np.zeros((min(batch_size, len(tag_list)), max_seq_len))
            for i in range(len(tag_list)):
                matrix[i] = [tagset[word] if word in tagset else UNKNOWN_INDEX for word in tag_list[i]] + [IGNORE_TAG_INDEX for i in range(max_seq_len - len(tag_list[i]))]
            batched_tag_indices.append(matrix)

            
        return batched_word_indices, batched_tag_indices, batched_lengths


In [10]:
def read_tagset(tag_file):
    """
    Utility function, loads tag file into a dictionary from tag string to tag index

    Arguments:
    - tag_file:   file location of the tagset

    Outputs:
    - tagset:     a dictionary mapping tag strings (e.g. "VB") to a unique index
    """
    tagset = {}
    with open(tag_file, encoding='utf8') as f:
        for line in f:
            columns = line.rstrip().split('\t')
            tag = columns[0]
            tag_id = int(columns[1])
            tagset[tag] = tag_id

    return tagset

In [11]:
def confusion_matrix(true, pred, num_tags):
    """
    Arguments:
    - true:       a list of true label values (integers)
    - pred:       a list of predicted label values (integers)
    - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

    Output: 
    - confusion_matrix:   a (num_tags x num_tags) matrix of integers

    confusion_matrix[i][j] = # predictions where true label
    was i and predicted label was j

    """

    confusion_matrix = np.zeros((num_tags, num_tags))

    #############################
    #       YOUR CODE HERE      #
    for i in range(len(true)):
        true_label = true[i]
        pred_label = pred[i]
        confusion_matrix[true_label][pred_label] += 1
    #############################


    return confusion_matrix



def precision(true, pred, num_tags):
    """
    Arguments:
    - true:       a list of true label values (integers)
    - pred:       a list of predicted label values (integers)
    - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

    Output: 
    - precision:  an array of length num_tags, where precision[i]
                gives the precision of class i

    Hints:  the confusion matrix may be useful
          be careful about zero division
    """

    precision = np.zeros(num_tags)

    #############################
    #       YOUR CODE HERE      #
    matrix = confusion_matrix(true, pred, num_tags)
    for k in range(num_tags):
        TP = matrix[k, k] #kth row: true = k
        FP = sum(matrix[:, k]) - matrix[k, k]
        precision[k] = TP / (TP + FP) if (TP + FP) != 0 else 0
    #############################



    return precision


def recall(true, pred, num_tags):
    """
    Arguments:
    - true:       a list of true label values (integers)
    - pred:       a list of predicted label values (integers)
    - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

    Output: 
    - recall:     an array of length num_tags, where recall[i]
                gives the recall of class i

    Hints:  the confusion matrix may be useful
          be careful about zero division
    """

    """
    YOUR CODE HERE
    """
    recall = np.zeros(num_tags)

    #############################
    #       YOUR CODE HERE      #
    matrix = confusion_matrix(true, pred, num_tags)
    for k in range(num_tags):
        TP = matrix[k, k] #kth row: true = k
        FN = sum(matrix[k, :]) - matrix[k, k]
        recall[k] = TP / (TP + FN) if (TP + FN) != 0 else 0
    #############################


    return recall


def f1_score(true, pred, num_tags):
    """
    Arguments:
    - true:       a list of true label values (integers)
    - pred:       a list of predicted label values (integers)
    - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

    Output: 
    - f1:         an array of length num_tags, where f1[i]
                gives the recall of class i
    """
    f1 = np.zeros(num_tags)

    #############################
    #       YOUR CODE HERE      #
    p = precision(true, pred, num_tags)
    r = recall(true, pred, num_tags)
    for k in range(num_tags):
        f1[k] = 2 * (p[k] * r[k]) / (p[k] + r[k])
    #############################


    return f1


In [21]:
def eval_per_class(model, dev_content, vocab, tagset):
    """
    Prints precision, recall, and F1 for each class in the tagset
    """
    # batch the data
    batched_idx, batched_tags, batched_lens = dev_content
    # compute idx --> tag from tag --> idx
    reverse_tagset = {v: k for k,v in tagset.items()}
    # evaluate model on hold-out set
    acc, true, pred = model.evaluate(batched_idx, batched_lens, batched_tags, tagset)
    true = np.array(true)
    pred = np.array(pred)
    # print(true[:200])
    # print(pred[:200])

    pr = precision(true, pred, len(tagset))
    re = recall(true, pred, len(tagset))
    f1 = f1_score(true, pred, len(tagset))

    for idx, tag in reverse_tagset.items():
        if tag == "B-PER" or tag == "I-PER":
            print("***********************")
            print("TAG: {}".format(tag))
            num_pred = np.sum(pred == idx)
            num_true = np.sum(true == idx)
            print("({} pred, {} true)".format(num_pred, num_true))

            print("PRECISION: \t{:.3f}".format(pr[idx]))
            print("RECALL: \t{:.3f}".format(re[idx]))
            print("F1 SCORE: \t{:.3f}".format(f1[idx]))

In [13]:
def combine(folder):
    filenames = [f for f in listdir(folder) if isfile(join(folder, f))]
    
    out = ""
    for i in range(len(filenames)):
        with open(folder + '/' + filenames[i], encoding='utf-8') as infile: 
            content = infile.read()
            out = out + content
        out = out + "\n"
    return out

In [16]:
# read the files
# all_books = combine('../data/correct_BIO_output')

# with open('../data/all_books.txt', 'w', encoding='utf-8') as outfile:
#     outfile.write(all_books) 

    
tagset = read_tagset('/NER_labels.txt')
with open("/all_books.txt" ,encoding='utf-8') as file:
  all_books = file.read()

dataset = Dataset(all_books)

BATCH_SIZE = 32 #for stocastic gradient descent purpose


# train_batch_idx, train_batch_tags, train_batch_lens = train_dataset.get_batches(BATCH_SIZE, vocab, tagset)
# dev_batch_idx, dev_batch_tags, dev_batch_lens = dev_dataset.get_batches(BATCH_SIZE, vocab, tagset)
# test_batch_idx, test_batch_tags, test_batch_lens = test_dataset.get_batches(BATCH_SIZE, vocab, tagset)

In [17]:
len(dataset.tags)

8867

In [22]:
#Perform K-fold Cross Validation and train the model
np.random.seed(159) 
shuffle = np.random.permutation(range(len(dataset.sentences)))

sentences = [dataset.sentences[i] for i in shuffle]
length = int(len(sentences) / 12)
tags = [dataset.tags[i] for i in shuffle]
dataset.tags_chunk = []
dataset.sentences_chunk = []


#Divide data into 10 chunks
for k in range(12):
    if k != 11:
        fold_sentences = sentences[length*k : length*(k + 1)]
        fold_tags = tags[length*k : length*(k + 1)]
        dataset.sentences_chunk.append(fold_sentences)
        dataset.tags_chunk.append(fold_tags)
    else:
        fold_sentences = sentences[length*k :]
        fold_tags = tags[length*k :]
        dataset.sentences_chunk.append(fold_sentences)
        dataset.tags_chunk.append(fold_tags)
#     print(len(fold_sentences))
        

#Loop 10 times, each time training with different chunks
dev_accuracy = []
HIDDEN_SIZE = 64
for k in range(10):
    train_batch_idx, train_batch_tags, train_batch_lens = dataset.get_batches(BATCH_SIZE, vocab, tagset, 0, 10, k)
    dev_batch_idx, dev_batch_tags, dev_batch_lens = dataset.get_batches(BATCH_SIZE, vocab, tagset, k, k + 1, -1)
    train_content = (train_batch_idx, train_batch_tags, train_batch_lens)
    dev_content = (dev_batch_idx, dev_batch_tags, dev_batch_lens)
    
    # intialize a new LSTMTagger model
    model = LSTMTagger(embeddings, HIDDEN_SIZE, len(tagset))
    # train the model
    acc, true, pred = model.run_training(train_content, dev_content, BATCH_SIZE, vocab, tagset,   
                       lr=5e-4, num_epochs=25, eval_every=5)
    dev_accuracy.append(acc)
    eval_per_class(model, dev_content, vocab, tagset)
    
    
test_batch_idx, test_batch_tags, test_batch_lens = dataset.get_batches(BATCH_SIZE, vocab, tagset, 10, 12, -1)

**** TRAINING *****
Epoch 0 | Loss: 101.0923843383789
Epoch 1 | Loss: 40.30803298950195
Epoch 2 | Loss: 32.02016830444336
Epoch 3 | Loss: 27.022197723388672
Epoch 4 | Loss: 23.046472549438477
**** EVALUATION *****
Dev Accuracy: 0.9630751358752022
**********************
Epoch 5 | Loss: 20.419544219970703
Epoch 6 | Loss: 18.28281021118164
Epoch 7 | Loss: 16.6927490234375
Epoch 8 | Loss: 15.398885726928711
Epoch 9 | Loss: 14.123956680297852
**** EVALUATION *****
Dev Accuracy: 0.97145583537319
**********************
Epoch 10 | Loss: 13.015137672424316
Epoch 11 | Loss: 12.283682823181152
Epoch 12 | Loss: 11.380812644958496
Epoch 13 | Loss: 10.458893775939941
Epoch 14 | Loss: 10.022538185119629
**** EVALUATION *****
Dev Accuracy: 0.9747334356719081
**********************
Epoch 15 | Loss: 9.26974105834961
Epoch 16 | Loss: 8.688904762268066
Epoch 17 | Loss: 8.359437942504883
Epoch 18 | Loss: 7.811026573181152
Epoch 19 | Loss: 7.23844051361084
**** EVALUATION *****
Dev Accuracy: 0.9756046965108

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in double_scalars


***********************
TAG: B-PER
(582 pred, 583 true)
PRECISION: 	0.837
RECALL: 	0.835
F1 SCORE: 	0.836
***********************
TAG: I-PER
(349 pred, 331 true)
PRECISION: 	0.722
RECALL: 	0.761
F1 SCORE: 	0.741
**** TRAINING *****
Epoch 0 | Loss: 97.98262786865234
Epoch 1 | Loss: 38.75595474243164
Epoch 2 | Loss: 30.78216552734375
Epoch 3 | Loss: 25.497711181640625
Epoch 4 | Loss: 22.060270309448242
**** EVALUATION *****
Dev Accuracy: 0.9689108152936673
**********************
Epoch 5 | Loss: 19.717912673950195
Epoch 6 | Loss: 17.78150749206543
Epoch 7 | Loss: 16.229705810546875
Epoch 8 | Loss: 14.970183372497559
Epoch 9 | Loss: 13.594632148742676
**** EVALUATION *****
Dev Accuracy: 0.976438565899715
**********************
Epoch 10 | Loss: 12.69483757019043
Epoch 11 | Loss: 11.72070598602295
Epoch 12 | Loss: 10.97913932800293
Epoch 13 | Loss: 10.384428977966309
Epoch 14 | Loss: 9.694375038146973
**** EVALUATION *****
Dev Accuracy: 0.9762684472419513
**********************
Epoch 15 | Lo

In [23]:
dev_accuracy

[0.9755632078994316,
 0.9745247309998724,
 0.974469160768453,
 0.9777851458885941,
 0.974377904199418,
 0.9746824078646137,
 0.9745595186935969,
 0.9793805612594113,
 0.9733862544391059,
 0.9764081523435758]

In [18]:
class LSTMTagger(nn.Module):
    """
    An LSTM model for sequence labeling

    Initialization Arguments:
    - embeddings:   a matrix of size (vocab_size, emb_dim)
                  containing pretrained embedding weights
    - hidden_dim:   the LSTM's hidden layer size
    - tagset_size:  the number of possible output tags

    """
    def __init__(self, embeddings, hidden_dim, tagset_size):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_labels = tagset_size

        #############################
        #       YOUR CODE HERE      #
        #############################

        # Initialize a PyTorch embeddings layer using the pretrained embedding weights
        vocab_size = len(embeddings)
        embedding_dim = len(embeddings[0])
        
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings.weight = nn.Parameter(embeddings)
        
        # Initialize an LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=0.3, num_layers=3)

        # Initialize a single feedforward layer
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)

    def forward(self, indices, lengths):
        """
        Runs a batched sequence through the model and returns output logits

        Arguments:
        - indices:  a matrix of size (batch_size x max_seq_len)
                    containing the word indices of sentences in the batch
        - lengths:  a vector of size (batch_size) containing the
                    original lengths of the sequences before padding

        Output:
        - logits:   a matrix of size (batch_size x max_seq_len x num_tags)
                    gives a score to each possible tag for each word
                    in each sentence 
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # cast arrays as PyTorch data types and move to GPU memory
        indices = torch.LongTensor(indices).to(device)
        lengths = torch.LongTensor(lengths).to(device)

        # convert word indices to word embeddings
        embeddings = self.embeddings(indices)

        # pack/pad handles variable length sequence batching
        # see here if you're curious: https://gist.github.com/HarshTrivedi/f4e7293e941b17d19058f6fb90ab0fec
        packed_input_embs = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted=False)
        # run input through LSTM layer
        packed_output, _ = self.lstm(packed_input_embs)
        # unpack sequences into original format
        padded_output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

        logits = self.hidden2tag(padded_output)
        return logits

    def run_training(self, train_content, dev_content, batch_size, vocab, tagset,
                         lr=5e-4, num_epochs=100, eval_every=5):
        """
        Trains the model on the training data with a learning rate of lr
        for num_epochs. Evaluates the model on the dev data eval_every epochs.

        Arguments:
        - train_dataset:  Dataset object containing the training data
        - dev_dataset:    Dataset object containing the dev data
        - batch_size:     batch size for train/dev data
        - vocab:          a dictionary mapping word strings to indices
        - tagset:         a dictionary mapping tag strings to indices
        - lr:             learning rate
        - num_epochs:     number of epochs to train for
        - eval_every:     evaluation is run eval_every epochs
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#         if str(device) == 'cpu':
#             print("Training only supported in GPU environment")
#             return

        # clear unreferenced data/models from GPU memory 
        torch.cuda.empty_cache()
        # move model to GPU memory
        self.to(device)

        # set the optimizer (Adam) and loss function (CrossEnt)
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        loss_function = nn.CrossEntropyLoss(ignore_index=-100)

        # batch training and dev data
        train_batch_idx, train_batch_tags, train_batch_lens = train_content
        dev_batch_idx, dev_batch_tags, dev_batch_lens = dev_content

        print("**** TRAINING *****")
        for i in range(num_epochs):
            # sets the model in train mode
            self.train()

            total_loss = 0
            for b in range(len(train_batch_idx)):
                # compute the logits
                logits = self.forward(train_batch_idx[b], train_batch_lens[b])
                # move labels to GPU memory
                labels = torch.LongTensor(train_batch_tags[b]).to(device)
                # compute the loss with respect to true labels
                loss = loss_function(logits.view(-1, len(tagset)), labels.view(-1))
                total_loss += loss
                # propagate gradients backward
                loss.backward()
                optimizer.step()
                # set model gradients to zero before performing next forward pass
                self.zero_grad()

            print("Epoch {} | Loss: {}".format(i, total_loss))

            if (i + 1) % eval_every == 0:
                print("**** EVALUATION *****")
                # sets the model in evaluate mode (no gradients)
                self.eval()
                # compute dev f1 score
                acc, true, pred = self.evaluate(dev_batch_idx, dev_batch_lens, dev_batch_tags, tagset)
                print("Dev Accuracy: {}".format(acc))
                print("**********************")
                
        return acc, true, pred

                
                
    def evaluate(self, batched_sentences, batched_lengths, batched_labels, tagset):
        """
        Evaluate the model's predictions on the provided dataset. 

        Arguments:
        - batched_sentences:  a list of matrices, each of size (batch_size x max_seq_len),
                              containing the word indices of sentences in the batch
        - batched_lengths:    a list of vectors, each of size (batch_size), containing the
                              original lengths of the sequences before padding
        - batched_labels:     a list of matrices, each of size (batch_size x max_seq_len),
                              containing the tag indices corresponding to sentences in the batch
        - num_tags:           the number of possible output tags

        Output:
        - accuracy:           the model's prediction accuracy
        - all_true_labels:    a flattened list of all true labels
        - all_predictions:    a flattened list of all of the model's corresponding predictions

        """

        all_true_labels = []
        all_predictions = []

        for b in range(len(batched_sentences)):
            logits = self.forward(batched_sentences[b], batched_lengths[b])
            batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()

            batch_size, _ = batched_sentences[b].shape

            for i in range(batch_size):
                tags = batched_labels[b][i]
                preds = batch_predictions[i]

                seq_len = int(batched_lengths[b][i])
                for j in range(seq_len):
                    all_predictions.append(int(preds[j]))
                    all_true_labels.append(int(tags[j]))


        acc = accuracy(all_true_labels, all_predictions)

        return acc, all_true_labels, all_predictions

In [19]:
def accuracy(true, pred):
    """
    Arguments:
    - true:       a list of true label values (integers)
    - pred:       a list of predicted label values (integers)

    Output:
    - accuracy:   the prediction accuracy
    """
    true = np.array(true)
    pred = np.array(pred)

    num_correct = sum(true == pred)
    num_total = len(true)

    return num_correct / num_total

In [ ]:
# np.random.seed(159)

# HIDDEN_SIZE = 64
# # intialize a new LSTMTagger model
# model = LSTMTagger(embeddings, HIDDEN_SIZE, len(tagset))
# # train the model
# model.run_training(train_dataset, dev_dataset, BATCH_SIZE, vocab, tagset,   
#                    lr=5e-4, num_epochs=25, eval_every=5)
# eval_per_class(model, dev_dataset, vocab, tagset)

In [ ]:
# eval_per_class(model, dev_dataset, vocab, tagset)